In [58]:
import open3d as o3d
import numpy as np
import copy as copy
import matplotlib.pyplot as plt

from rotation import rotation
from points_Octree import points_Octree
from Voxel_Octree import Voxel_Octree
from parcourir_octree_dico import dic

from collision_octree import collision_octree
from collision_octree_non_dic import collision_octree_non_dic

from mask_tab import mask_tab
from Voxelize import Voxelize
from coord_voxel import coord_voxel



tool = o3d.io.read_triangle_mesh('../Object/Deposition Head.stl')
collision = o3d.io.read_triangle_mesh('../Object/collision.stl')


# Creation voxel à partir des meshs

In [28]:
voxel_size = 0.5  # Vous pouvez définir la taille des voxels en fonction de vos besoins
voxel_grid1 = o3d.geometry.VoxelGrid.create_from_triangle_mesh(tool, voxel_size)
voxel_grid2 = o3d.geometry.VoxelGrid.create_from_triangle_mesh(collision, voxel_size)

In [29]:
visualisation([voxel_grid1])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# Creation voxel à partir d'un nuage de points

In [30]:
voxel_size = 0.5  # Taille du voxel
number_of_points = 1000000
point_cloud = tool.sample_points_uniformly(number_of_points)

In [31]:
vect = - point_cloud.get_min_bound() + voxel_size/2

In [32]:
point_cloud.translate(vect)

PointCloud with 1000000 points.

In [33]:
voxel_grid1 = o3d.geometry.VoxelGrid.create_from_point_cloud(point_cloud, voxel_size)

In [34]:
voxel_grid1.get_min_bound()

array([0., 0., 0.])

In [35]:
visualisation([voxel_grid1,tool])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [36]:
coord_voxel(voxel_grid1).min(axis=0)

array([0, 0, 0], dtype=int32)

# Récuperation des voxels de la grille 

In [37]:
grid_indices = coord_voxel(voxel_grid1)

In [38]:
len(grid_indices)

139868

In [39]:
test = np.array([np.asarray(voxel_grid1.get_voxel_bounding_points(np.array(ele))) for ele in grid_indices])

In [40]:
test=test.reshape((len(grid_indices)*8,3))

In [41]:

unique_tab = np.unique(test, axis=0)

len(unique_tab)


294711

# Creation octree

In [44]:
voxel_size=0.5
delta_max_deth=2
octree = Voxel_Octree(voxel_grid1 , delta_max_deth , voxel_size)

In [45]:
octree.get_center()

array([80.25, 80.25, 80.25])

In [46]:
octree.locate_leaf_node(np.array([ 31.5,  26.5,   0. ]))

(OctreeColorLeafNode with color [0, 0, 0],
 OctreeNodeInfo with origin [30.0938, 25.0781, 0], size 2.50781, depth 6, child_index 0)

In [47]:
visualisation([octree])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [48]:
points = coord_voxel(voxel_grid1)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

max_depth= np.floor(np.log(pcd.get_max_bound().max())/np.log(2)).astype(int)

octree = o3d.geometry.Octree(max_depth)
octree.convert_from_point_cloud(pcd)
visualisation([octree])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [49]:
octree.get_center()

array([ 59.6,  59.6, 161.6])

In [50]:
dictionnaire={}
dic(octree,dictionnaire)

In [53]:
dictionnaire[(0,(-102.0, -102.0, 0.0))]

{'infos': (0, array([-102., -102.,    0.]), 323.2),
 'enfants': [(1, (-102.0, -102.0, 0.0)),
  (1, (59.599999999999994, -102.0, 0.0)),
  (1, (-102.0, 59.599999999999994, 0.0)),
  (1, (59.599999999999994, 59.599999999999994, 0.0)),
  (1, (-102.0, -102.0, 161.6)),
  (1, (59.599999999999994, -102.0, 161.6)),
  (1, (-102.0, 59.599999999999994, 161.6)),
  (1, (59.599999999999994, 59.599999999999994, 161.6))]}

# Collision octrees

- exemple 1 : 3 collisions

In [60]:
# Création d'un nuage de points simple pour l'octree1
points1 = np.array([
    [0, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [1, 0, 0]
], dtype=np.float64)

pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(points1)
pcd1.colors = o3d.utility.Vector3dVector(np.full((len(points1), 3), [0, 0, 0]))

# Création d'un nuage de points simple pour l'octree2
points2 = np.array([
    [0.5, 0.5, 0.5],
    [1, 1, 1],
    [0, 0, 1.5],
    [1, 1, 0]
], dtype=np.float64)

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(points2)
pcd1.colors = o3d.utility.Vector3dVector(np.full((len(points2), 3), [1, 0, 0]))

# Construction des octrees à partir des nuages de points
voxel_size = 0.5
octree1 = o3d.geometry.Octree(2)  # max_depth = 2
octree1.convert_from_point_cloud(pcd1, voxel_size)

octree2 = o3d.geometry.Octree(2)  # max_depth = 2
octree2.convert_from_point_cloud(pcd2, voxel_size)

In [61]:
visualisation([octree1 , octree2])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [73]:
collision  = collision_octree_non_dic(octree1, octree2)
len(collision)

3

# Rotation

In [67]:
tool = o3d.io.read_triangle_mesh('../Object/Deposition Head.stl')
theta, phi = np.deg2rad(45) , np.deg2rad(45)
tool_r = rotation(tool , theta, phi )

visualisation([tool,tool_r])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [68]:
voxel_size, number_of_points = 0.5 , 1000000
tool_voxel = Voxelize(tool , voxel_size , number_of_points)
tool_r_voxel = Voxelize(tool_r , voxel_size , number_of_points)

tool_voxel_coord = coord_voxel(tool_voxel)
tool_r_voxel_coord = coord_voxel(tool_r_voxel)

# Visualisation

In [74]:
def visualisation(see_what : list):
    create_coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])
    see_what.append(create_coordinate_frame)
    o3d.visualization.draw_geometries(see_what, mesh_show_wireframe=False )

In [85]:
tool = o3d.io.read_triangle_mesh('../Object/Deposition Head.stl')
tool_r_list=[] #ok
tool_r_voxel_list=[] #ok
tool_r_voxel_coord_list=[]
tool_r_voxel_coord_change_list=[] #ok
voxel_size=0.5
number_of_points=100000
for j in Circle:
    for ind, delta in enumerate((Circle[j])):
        tool_r = rotation(tool,delta ,j*np.deg2rad(theta)) #tool_path[0]+np.array([0,0,6])
        
        tool_r_voxel = Voxelize(tool_r , voxel_size , number_of_points)
        
        voxel_r_voxel_coord= coord_voxel(tool_r_voxel)
        
        voxel_r_voxel_coord = tool_r_voxel.origin + tool_r_voxel.voxel_size * voxel_r_voxel_coord

        
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(world_coord)
visualisation(tool_list)

NameError: name 'world_coord' is not defined

In [79]:
tool_r_voxel_coord_change_pcd_list=[]
for ele in tool_r_voxel_coord_change_list : 
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(ele)
    tool_r_voxel_coord_change_pcd_list.append(pcd)

In [80]:
visualisation(tool_r_voxel_coord_change_pcd_list+tool_r_list)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [81]:
tool = o3d.io.read_triangle_mesh('../Object/Deposition Head.stl')



In [82]:
tool_r.get_min_bound()

array([-21.6559425 , -30.32660979,  -0.26396456])

In [83]:
visualisation([tool,tool_r])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
